In [ ]:
!pip install --upgrade setuptools




In [ ]:
!pip cache purge

Files removed: 4


In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
from matplotlib import pyplot  as plt
from google.colab import files
files.upload()

In [ ]:
rets = pd.read_csv('FINAL-N50-NN50-N500_Hist-data--EPS-Buffett-detrend-repo-PROCESSED.csv')
rets.tail()

,indexid,Date,pe,pb,divyield,price,Mva-90,Mva-30,up52wk,dn52wk,...,Eps-lag-270,Eps-lag-365,actual_ret,ret5yr,ret3yr,GDP,Index2GDP,Ind2GDP-detrended,rr,repo_rate
19578,4,45372,25.49,4.73,1.23,58918.3,56447.788889,58998.4475,0.593149,0.023301,...,1510.169082,1444.744878,NaN,NaN,NaN,NaN,NaN,NaN,6.5,6.5
19579,4,45373,25.60,4.75,1.24,59188.9,56562.787302,59000.2050,0.600466,0.018815,...,1510.013750,1445.029014,NaN,NaN,NaN,NaN,NaN,NaN,6.5,6.5
19580,4,45377,25.87,4.80,1.22,59801.5,56679.718254,59030.3525,0.617030,0.008660,...,1510.342277,1444.796232,NaN,NaN,NaN,NaN,NaN,NaN,6.5,6.5
19581,4,45378,25.92,4.81,1.21,59911.7,56794.503968,59068.4050,0.620010,0.006833,...,1510.168813,1445.022945,NaN,NaN,NaN,NaN,NaN,NaN,6.5,6.5
19582,4,45379,22.43,4.31,1.33,60624.3,56916.094444,59180.2550,0.639279,0.000000,...,1510.336742,1445.009630,NaN,NaN,NaN,NaN,NaN,NaN,6.5,6.5


In [ ]:
from __future__ import absolute_import, division, print_function
import pathlib
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
print(tf.__version__)

2.15.0


In [ ]:
#rets = rets[rets.indexid < 3.6]

In [ ]:
rets.sort_values(by=['Date'], inplace=True)
#rets = rets.drop(['Date','pe-30','pb-30', 'dy-30', 'Mva-30', 'price', 'EPS-mva90','ret3yr', 'GDP', 'repo_rate'], axis=1)
rets = rets.drop(['Date','pe-30','pb-30', 'dy-30', 'Mva-30', 'price', 'EPS-mva90','ret3yr', 'GDP', 'repo_rate'], axis=1)
rets_orig = rets.drop('ret5yr', axis=1)
rets.dropna(inplace=True)
rets.head()


In [ ]:
rets.describe()


In [ ]:
#sns.pairplot(rets[["ret5yr", "rr" ]], diag_kind="kde")
#sns.pairplot(rets[["ret5yr", "pe", "pb", "divyield", "up52wk", "dn52wk","Index2GDP" ]], diag_kind="kde")

In [ ]:
test_2 = rets.iloc[12500:,:]
rets = rets.iloc[:12500,:]
print(test_2.shape, rets.shape)

(1331, 17) (12500, 17)


In [ ]:
rets_orig.dropna(subset = ['indexid', 'pe', 'pb', 'divyield', 'Mva-90','up52wk', 'dn52wk', 'Eps-ttm', 'Eps-lag-90', 'Eps-lag-180', 'Eps-lag-270', 'Eps-lag-365','Index2GDP', 'Ind2GDP-detrended', 'rr' ], inplace=True)
rets_orig.reset_index()
rets_orig_actuals = rets_orig.pop('actual_ret')
rets_orig.head()

In [ ]:
rets_orig.shape

In [ ]:
# Only used in case we do not want to split train and test (because we have already got test set: test_2)
#train_set = rets.copy()
#train_set.tail()

In [ ]:
from sklearn.model_selection import ShuffleSplit
#from sklearn.model_selection import train_test_split
#train_set, test_set, train_labels, test_labels =  train_test_split(rets, rets_labels, test_size=0.2, shuffle=False)
split = ShuffleSplit(n_splits = 1, test_size = 0.05, random_state = 0)
#["ind_wkyr"] = y8r["index_id"].astype(str) + y8r["weekyear"].astype(str)
#y8r = y8r.drop("index_id")
#y8r = y8r.drop("weekyr")
for train_ind, test_ind in split.split(rets):
    train_set = rets.iloc[train_ind]
    test_set = rets.iloc[test_ind]

In [ ]:
train_set.pop('ret5yr')
test_set.pop('ret5yr')
test_2.pop('ret5yr')
#test_actuals = test_2.pop('actual_ret')
train_labels = train_set.pop('actual_ret')
test_labels = test_set.pop('actual_ret')
test_2_labels = test_2.pop('actual_ret')

In [ ]:
#test_2_labels = test_actuals.copy()
#test_2_labels.tail()

In [ ]:
test_2_labels.tail()

In [ ]:
#train_labels = train_set.pop("ret5yr")
#train_labels = train_set.pop("ret3yr")
train_set.tail()


In [ ]:
#test_labels = test_set.pop("ret5yr")
#test_labels = test_set.pop("ret3yr")
test_set.tail()

In [ ]:
#test_set.pop('up52wk')
#test_set.pop('dn52wk')
#train_set.pop('up52wk')
#train_set.pop('dn52wk')

In [ ]:
train_stats = train_set.describe()
#train_stats.pop('ret8yr')
train_stats = train_stats.transpose()
print(train_stats.head)
print(train_set.shape)


In [ ]:
#test_2_labels = test_2.pop("ret5yr")
#test_2_labels = test_2.pop("ret3yr")
test_2.tail()

In [ ]:
test_2_labels.describe()

In [ ]:
def norm(x):
  return (x - train_stats['mean'])/train_stats['std']
normed_train_set = norm(train_set)
normed_test_set = norm(test_set)
normed_test_2 = norm(test_2)

In [ ]:
def my_model():
  model = keras.Sequential([
      layers.Dense(128, activation=tf.nn.relu, input_shape=[len(normed_train_set.keys())]),
      layers.Dense(128, activation= tf.nn.relu),
      layers.Dense(64, activation= tf.nn.relu),
      #layers.Dense(64, activation= tf.nn.relu),
      layers.Dense(32, activation= tf.nn.relu),
      layers.Dense(1, activation= tf.nn.relu)
  ])

  opt = tf.keras.optimizers.Adam(.001)
  model.compile(loss='mse',
                optimizer=opt, metrics=['mae', 'mse'])
  return model


In [ ]:
model = my_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               2048      
                                                                 
 dense_21 (Dense)            (None, 128)               16512     
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 32)                2080      
                                                                 
 dense_24 (Dense)            (None, 1)                 33        
                                                                 
Total params: 28929 (113.00 KB)
Trainable params: 28929 (113.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Train model

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print(" ")
    print('.',end='')

EPOCHS = 700
history = model.fit(
    normed_train_set, train_labels,
    epochs=EPOCHS, validation_split=0.2, verbose=0,
    callbacks=[PrintDot()]
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('MAE(rets)')
  plt.plot(hist['epoch'], hist['mae'], label="Train Error")
  plt.plot(hist['epoch'], hist['val_mae'], label="Validation Error")
  plt.legend()
  plt.ylim([0,5])

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('MSE($rets^2$)')
  plt.plot(hist['epoch'], hist['mse'], label="Train Error")
  plt.plot(hist['epoch'], hist['val_mse'], label="Validation Error")
  plt.legend()
  plt.ylim([0,20])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_set, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f}x, {:5.2f}x".format(mae, mse))


In [ ]:
test_predictions = model.predict(normed_test_set).flatten()
plt.scatter(test_labels,test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predicted values')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100,100],[-100,100])

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=50)
plt.xlabel("Prediciton Error(x)")
_ = plt.ylabel("Count")

In [ ]:
test_2.shape, normed_test_2.shape

((1331, 15), (1331, 15))

In [ ]:
loss, mae, mse = model.evaluate(normed_test_2, test_2_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f}x, {:5.2f}x".format(mae, mse))
#print("Testing set Mean Abs Error: {:5.2f} x".format(mae))
#print("Testing set Mean sqrd Error: {:5.2f} x".format(mse))

42/42 - 0s - loss: 0.0851 - mae: 0.2215 - mse: 0.0851 - 478ms/epoch - 11ms/step
Testing set Mean Abs Error:  0.22x,  0.09x


In [ ]:
#rets_orig.sort_index(inplace=True)
#rets_orig_actuals.sort_index(inplace=True)
normed_orig = norm(rets_orig)
normed_orig.shape, rets_orig.shape

((17211, 15), (17211, 15))

In [ ]:

all_predictions = model.predict(normed_orig).flatten()

538/538 [==============================] - 1s 2ms/step


In [ ]:
test_2_predictions = model.predict(normed_test_2).flatten()
plt.scatter(test_actuals,test_2_predictions)
plt.xlabel('True Values')
plt.ylabel('Predicted values')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100,100],[-100,100])

In [ ]:
error2 = test_2_predictions - test_2_labels
plt.hist(error2, bins=50)
plt.xlabel("Prediciton Error(x)")
_ = plt.ylabel("Count")

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_labels.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_labels.csv")

In [ ]:
test_set.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_set.csv")

In [ ]:
dataset = pd.DataFrame({'ret': test_predictions[:]})
dataset.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_predictions.csv")

In [ ]:
dataset = pd.DataFrame({'ret': test_2_predictions[:]})
dataset.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_2_predictions.csv")

In [ ]:
dataset = pd.DataFrame({'ret': all_predictions[:]})
dataset.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/all_predictions.csv")

In [ ]:
test_2_labels.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_2_labels.csv")
test_2.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_2.csv")


In [ ]:
test_actuals.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/test_2_actual_rets.csv")

In [ ]:
rets_orig.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/rets_orig.csv")
rets_orig_actuals.to_csv("drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/rets_orig_actuals.csv")

In [ ]:
#Load already saved model
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/models/model5yr-All-800EPOCH-25VAL-128-128-64-32-1/")
model.summary()